In [ ]:
# -*- coding: utf-8 -*-
from poke_env.player.env_player import Gen8EnvSinglePlayer

class SimpleRLPlayer(Gen8EnvSinglePlayer):
    def embed_battle(self, battle):
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = move.base_power / 100 # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                )

        # We count how many pokemons have not fainted in each team
        remaining_mon_team = len([mon for mon in battle.team.values() if mon.fainted]) / 6
        remaining_mon_opponent = (
            len([mon for mon in battle.opponent_team.values() if mon.fainted]) / 6
        )

        # Final vector with 10 components
        return np.concatenate(
            [moves_base_power, moves_dmg_multiplier, [remaining_mon_team, remaining_mon_opponent]]
        )

    def compute_reward(self, battle) -> float:
        return self.reward_computing_helper(
            battle,
            fainted_value=2,
            hp_value=1,
            status_value=0.2,
            victory_value=30,
        )

In [ ]:
import asyncio
import numpy as np
from poke_env.utils import to_id_str
import abc
from tf_agents.environments import py_environment
from tf_agents.specs import array_spec, BoundedArraySpec
class PokeEnv(py_environment.PyEnvironment):
    def __init__(self, player_const, opponent_const) -> None:
        #self.poke_env = poke_env_creatobr(battle_format="gen8randombattle")
        self.player = player_const()
        self.opponent = opponent_const()
        print(type(self.player.action_space))
        print(self.player.action_space)
        self._action_spec = BoundedArraySpec(
            shape=(), 
            dtype=np.int32, 
            minimum=0, 
            maximum=len(self.player.action_space)-1, 
            name='action')
        
        self._observation_spec = BoundedArraySpec(
            shape=(10,), 
            dtype=np.float32)

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def action_spec(self):
        return self._action_spec
    
    def _reset(self):
        return self.poke_env.reset()
    
    def _step(self, action):
        return self.poke_env.step(action)
   

In [ ]:
from tf_agents.environments import validate_py_environment
player = SimpleRLPlayer(battle_format="gen8randombattle")
from poke_env.player.random_player import RandomPlayer

player.play_against(opponent=RandomPlayer(battle_format="gen8randombattle"))

environment = PokeEnv(player)
validate_py_environment(environment, episodes=5)


In [2]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common


In [4]:

fc_layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(22)
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

ValueError: No known conversion from type `<class 'int'>` to a TensorSpec.  Saw:
  22